<a href="https://colab.research.google.com/github/MichaelSeitz98/Seminararbeit_Kleinkinderkennung/blob/main/training_kinddetektion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Verbindung zu Google Drive herstellen. 

Für Zugriff auf Datensatz, zum Speichern der Modelle, des Trainingsfortschrittes und der Trainingseinstellungen.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Vorbereitung vor jedem neuen Trainingslauf

Es wird ein neuer Ordner erstellt, in welchem später alle benötigten Dateien, die Trainingseinstellungen und die trainierten Modelle abgespeichert werden. Der Name von training_folder vor jedem neuen Trainingslauf anpassen. 

In [ ]:
training_folder = '/content/drive/MyDrive/training02_ssdMobileNet_ds1024Erw_8_20000_100'

%mkdir {training_folder}

Hier wird der Speicherpfad des verwendeten Datensatzes gesetzt. 

In [ ]:
datensatz_folder = '/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px_mit_Erw'

Für ein Trainingslauf werden die folgenden Datein benötigt: 

 1. train.record
 2. test.record 
 3. test_labels.csv
 4. train_labels.csv
 5. Configurationsdatei des Modells
 6. labelMap 
 7. generateTF_record Skript

Diese Dateien werden aus Gründen der Nachvollziebarkeit immer im training_folder mit abgelegt. Auch wenn z.B. bei der LabelMap keine trainingsspezifischen Änderung anfallen, wird es so ermöglicht, dass immer alle Dateien zusammen und nachvollziebar im Ordner abgelegt sind. 

## Installation Tensorflow Object Detection API

Da die Tensorflow Object Detection API nicht auf Colab vorinstalliert ist. Muss diese bei jeder neuen Laufzeit neu installiert werden.
Dies geschieht in meheren Schritten. 

Zuerst wird Tensorflow installiert. Aufgrund verschiedener Installationsprobleme hat sich die Version 2.6.0 für mich als zuverlässigste herausgestellt. 

In [ ]:
!pip install tensorflow=="2.6.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564.4 MB 2.8 kB/s 
     |████████████████████████████████| 132 kB 4.9 MB/s 
     |████████████████████████████████| 14.8 MB 44.0 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=27e4fbe4661104187fc10cb1ff8fd94ee328c53db6541e385dc1f347028d452d
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=2e963a75fc4f04e9bbee036f0782b540974855e3b31210b6d6d2e9cc23bb44a2
  Stored in directory: /root/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68716 sha256=82f6e03f901613b5b635a7a97815e3a8fab6cb3998c910cfb9ce5af519436c1f
  Stored in directory: /root/.cache/pip/wheels/62/7

Wenn noch nicht vorhanden, dann werden (via git clone) das tensorflow/models repository von Github geklont. Einiger der Dateien daraus werden für die tatsächliche Installation benötigt (im nächsten Schritt).  

In [ ]:
import pathlib
import os
 
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3598, done.
remote: Counting objects: 100% (3598/3598), done.
remote: Compressing objects: 100% (2996/2996), done.
remote: Total 3598 (delta 948), reused 1523 (delta 548), pack-reused 0
Receiving objects: 100% (3598/3598), 47.09 MiB | 32.69 MiB/s, done.
Resolving deltas: 100% (948/948), done.


Da eine parallele Überwachung von Trainingsmetriken (loss) und Validierungs-Metriken (loss, map, usw.) nicht über colab möglich ist, evaluiere ich das Modell nach abgeschlossenem Trainingslauf. Dazu müssen zwei Anpassungen an dieser Stelle vorgenommen werden. 

* Anzahl der per default gepeicherten Checkpoints erhöhen. Statt 7 checkpoints nun bis zu 200 Checkpoints gespeichert. Anpassung in model_lib_v2, Zeile 443
* Anpassung, dass alle gespeicherten Checkpoints ausgewertet werden (und nicht nur einer), model_lib_v2 ,  Z.1135

nach: https://thachngoctran.medium.com/make-tensorflows-object-detection-validation-a-true-post-process-c45785f08d3a


Nach diesr Anpassung erfolgt die eigentliche Installation der Tensorflow-Object Detection API (davor noch setup.py Datei kopieren).

In [ ]:
%%bash
cd /content/models/research
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696664 sha256=6ac61948548ba07f77f80515255fcbbf7d26776a1d4a4f4988872003ca9134f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-hy68h7dz/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=f5d91d8b79d15ae08beba1bbcb4689c73e2ffae928ae727f90db0c48c9366b1f
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44009 sha256=9ace6744d6d99f8c233b89e14c545c24dc19ac89cce7c7b01fa39286a0915ab0
  Stored in directory: /root/.cache/pip/wheels/d6/e5/b1/6b151d9b535ee50aaa6ab27d145a0104b6df02e5636f0376da
  Created wheel for docopt: filename=docopt-

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


Nun ist die Installation der Tensorflow OD API abgeschlossen und kann mit dem *model_builder_tf2_test.py* Skript getestet werden. 

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

## Datenvorbereitung für Training

Nun werden alle für das Training benötigten Dateien erzeugt, geladen oder angepasst.

In [ ]:
%cd {training_folder}

/content/drive/MyDrive/training02_ssdMobileNet_ds1024Erw_8_20000_100


Das *generate_tfrecord.py* Skript habe ich für die Kleinkind-Detektion angepasst und auf meinen GitHub-Account abgelegt. Von dort wird es nun via wget in den Trainingsordner geladen. 

In [ ]:
!wget https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/44688230a2a4692e06b2ab927ab7abca0eb163ec/generate_tfrecord.py

--2022-11-30 10:07:01--  https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/44688230a2a4692e06b2ab927ab7abca0eb163ec/generate_tfrecord.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3394 (3.3K) [text/plain]
Saving to: ‘generate_tfrecord.py’

generate_tfrecord.p 100%[===================>]   3.31K  --.-KB/s    in 0.001s  

2022-11-30 10:07:02 (2.55 MB/s) - ‘generate_tfrecord.py’ saved [3394/3394]



Auch die angepasste *labelmap.pbtxt* wird von meinem (öffentlichem) GitHub-Repository geladen.

In [ ]:
!wget https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/main/labelmap.pbtxt

--2022-11-30 10:07:02--  https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/main/labelmap.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37 [text/plain]
Saving to: ‘labelmap.pbtxt’

labelmap.pbtxt      100%[===================>]      37  --.-KB/s    in 0s      

2022-11-30 10:07:02 (1.44 MB/s) - ‘labelmap.pbtxt’ saved [37/37]



Nun werden die Bilder und Labels in den *training_folder* kopiert.
Für jeden Data Split bzw. jedes Data set müssen die Labels, welche als *xml*-Datein vorliegen in eine **gemeinsame csv-Datei** überführt werden. Ich habe die Konvertierung bereits lokal durchgeführt und die csv-Datei mit allen Labeln bereits in den Datensatz-Order abgelegt. Aus Gründen der Nachvollziehbarkeit werde ich diese nun ebenfalls in den Trainingsordner kopieren.

In [ ]:
print(f'Datensatz Pfad: {datensatz_folder}')

train_labels_path = datensatz_folder + '/train_labels.csv'
validation_labels_path = datensatz_folder + '/validation_labels.csv'

!cp {train_labels_path} {training_folder}
!cp {validation_labels_path} {training_folder}

print (f'Training labels: {train_labels_path}')
print (f'Validation labels: {validation_labels_path}')

Datensatz Pfad: /content/drive/MyDrive/DS2_POOLG_80_20_20_1024px_mit_Erw
Training labels: /content/drive/MyDrive/DS2_POOLG_80_20_20_1024px_mit_Erw/train_labels.csv
Validation labels: /content/drive/MyDrive/DS2_POOLG_80_20_20_1024px_mit_Erw/validation_labels.csv


Hier wird aus den Labels und den Bildern eine Datei im tf-record-Format erstellt. Dieses Dateiformat ist für Tensorflow benötigt und gilt als besonders speichereffizient. Kann bis zu 5 Minuten dauern.  

In [ ]:
print(f'{datensatz_folder}/train')
print(f'{datensatz_folder}/validation')
!python generate_tfrecord.py --csv_input={train_labels_path} --image_dir={datensatz_folder}/train --output_path=train.record
!python generate_tfrecord.py --csv_input={validation_labels_path} --image_dir={datensatz_folder}/validation --output_path=validation.record

/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px_mit_Erw/train
/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px_mit_Erw/validation
2022-11-30 10:08:55.360378: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 10:08:55.512624: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-30 10:08:56.305846: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: 

Nun sind alle für das Training erforderlichen Datein im *training_folder* vorhanden. Jetzt werden die Dateipfade aller fürs Training relevanten Dateien werden gesetzt und ausgegeben.

In [ ]:
train_record_path = training_folder + '/train.record'
validation_record_path =  training_folder + '/validation.record'
labelmap_path =  training_folder + '/labelmap.pbtxt'

print (f'Diese Dateien müssen vorhanden sein in: {training_folder}\n')
print (f'train_record_path: {train_record_path}')
print (f'validation_record_path: {validation_record_path}')
print (f'labelmap_path: {labelmap_path}')

Diese Dateien müssen vorhanden sein in: /content/drive/MyDrive/training02_ssdMobileNet_ds1024Erw_8_20000_100

train_record_path: /content/drive/MyDrive/training02_ssdMobileNet_ds1024Erw_8_20000_100/train.record
validation_record_path: /content/drive/MyDrive/training02_ssdMobileNet_ds1024Erw_8_20000_100/validation.record
labelmap_path: /content/drive/MyDrive/training02_ssdMobileNet_ds1024Erw_8_20000_100/labelmap.pbtxt


## Modellauswahl und Trainingsparameter festlegen.

Im Tensorflow Object Dectetion Model Zoo stehen über 50 Modelle zur Verfügung, die auf dem COCO Datensatz vortrainiert wurden. Dabei sind die verschiednene Modellarchitekturen nach den Kriterien Genauigkeit und Geschwindigkeit hier aufgeslistet: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

Zuerst wird der Name des ausgewählten Models gesetzt und die gewünschten Trainingseinstellungen.

In [ ]:
batch_size = 8
num_steps = 20000
num_eval_steps = 100
nr_of_classes = 2

In [ ]:
name_model= 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
name_model_als_tar_gz = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

Nun kann das Modell über *wget* aus dem Tensorflow Model Zoo geladen werden

In [ ]:
# Vortrainiertes Modell aus TF Model Zoo laden. An dieser Stelle Modelle austauschen! 
#!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
#!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
#!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz



#download_link_model = f'http://download.tensorflow.org/models/object_detection/tf2/20200711/{name_model_als_tar_gz}'
download_link_model = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
print(download_link_model)
!wget {download_link_model}

#print (f'Download link {download_link_model}' )

http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
--2022-11-30 10:11:36--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 209.85.146.128, 2607:f8b0:4001:c1f::80
Connecting to download.tensorflow.org (download.tensorflow.org)|209.85.146.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  84.0MB/s    in 0.2s    

2022-11-30 10:11:36 (84.0 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



Das Modell wurde als tar.gz heruntergeladen und muss entzippt werden. Nach dem Entzippen benötige ich die *tar.gz*-Datei nicht mehr benötigt. Aus Gründen der  Übersichtlichkeit lsöche ich die Datei nach dem Entpacken wieder.

In [ ]:
%pwd

'/content/drive/MyDrive/training02_ssdMobileNet_ds1024Erw_8_20000_100'

In [ ]:
print (f'Entpacke Datei: {name_model_als_tar_gz}')
!tar xf {name_model_als_tar_gz}

Entpacke Datei: ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz


In [ ]:
!rm {name_model_als_tar_gz}

Ausgangspunkt auf ein vortrainiertes Modell setzen



In [ ]:
fine_tune_checkpoint = f'{training_folder}/{name_model}/checkpoint/ckpt-0'
print(fine_tune_checkpoint)

/content/drive/MyDrive/training02_ssdMobileNet_ds1024Erw_8_20000_100/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0


Jetzt lade ich die Konfigurationseinstellungen für das Training herunter. Dieser werden ebenfalls über GitHub via *wget* geladen. Alternativ kann dazu auch *pipeline.config* aus dem Ordner entpackten Modell-Ordner verwendet und angepasst werden..

In [ ]:
# !wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d1_640x640_coco17_tpu-8.config
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
#!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config
#wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/{name_model}.config

config_path = f'{name_model}.config'    # bei allem außer EfficientDet funktionierend
#config_path = '/content/drive/MyDrive/training3_efficientDetD1_ds2_8_20000_100/ssd_efficientdet_d1_640x640_coco17_tpu-8.config' # bei EfficientDet

# print (f'Config path gesetzt auf: {config_path}')

--2022-11-30 10:14:49--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4684 (4.6K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config’

ssd_mobilenet_v2_fp 100%[===================>]   4.57K  --.-KB/s    in 0.001s  

2022-11-30 10:14:49 (5.88 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config’ saved [4684/4684]



In [ ]:
#Nur für Faster-R-CNN benötigt
# config_path = '/content/drive/MyDrive/training3_faster_rcnn_ds2_8_20000_100/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.config'
# print (f'Config Path gesetzt auf: {config_path}')

Config Path gesetzt auf: /content/drive/MyDrive/training3_faster_rcnn_ds2_8_20000_100/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.config


In [ ]:
config_path= '/content/drive/MyDrive/training01_effDetD0_ds1024Erw_8_20000_100/ssd_efficientdet_d0_512x512_coco17_tpu-8.config'

*Config*-Datei wird mit den gewünschten Parametern befüllt. Weitere Anpassung manuell in der Config-Datei durchführen

In [ ]:
import re
with open(config_path) as f:
    config = f.read()

with open(config_path, 'w') as f:
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(validation_record_path), config)
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(nr_of_classes), config)
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [ ]:
%pwd

'/content/drive/MyDrive/training02_ssdMobileNet_ds1024Erw_8_20000_100'

In [ ]:
print(config_path)
%cat {config_path}

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 2
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    a

## Trainingslauf durchführen

Nun kann das Training gestartet werden.

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={config_path} \
    --model_dir={training_folder} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps} \
    --checkpoint_every_n=1000

## Evaluation nach abgeschlossenem Training

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py --model_dir={training_folder} --pipeline_config_path={config_path} --checkpoint_dir={training_folder}

## Export des trainierten Modell

In [ ]:
output_directory = training_folder+'/inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {training_folder} \
    --output_directory {output_directory} \
    --pipeline_config_path {config_path}

## Anwendung des trainierten Model

Am besten im eigenen Inferenz-Notebook anwenden: https://colab.research.google.com/drive/1NF6mlmV5zJGwR1Wh2UwTeOWHoeAENa3P?usp=sharing 


In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

In [ ]:
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'{output_directory}/saved_model')

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
#model_dir = '/content/drive/MyDrive/training_effDet_d0_4_3000_100/'
#print (model_dir)
#speicherPfadEvalBilder = model_dir + "angewendeteBilder"

#%mkdir {speicherPfadEvalBilder}
print(training_folder)

/content/drive/MyDrive/training3_faster_rcnn_ds2_8_20000_100


In [ ]:
print (training_folder)
speicherPfadEvalBilder = training_folder + "/angewendeteBilder"

%mkdir {speicherPfadEvalBilder}

for image_path in glob.glob('/content/drive/MyDrive/Child_Dataset/testEval/*g'):

  image_name = os.path.basename(image_path).replace(".jpg", "")
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(model, image_np)
  outImg = vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      min_score_thresh=0.5,
      max_boxes_to_draw=20,
      line_thickness=3)
  
  im = Image.fromarray(outImg)
  display(Image.fromarray(image_np))
  print ('Versuche hier zu speichern:' + speicherPfadEvalBilder + "/" + image_name + "_boxed.jpg")
  im.save(speicherPfadEvalBilder + "/" + image_name + "_boxed.jpg")
